In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'png'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L1VzZXJzL21waWVrZW5icm9jay9wZWVreGMuZ2l0aHViLmlv'
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

{"/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/importlib/_bootstrap.py": 1718724481.265324, "/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/importlib/_bootstrap_external.py": 1718724481.272269, "/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/zipimport.py": 1718724478.785079, "/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/codecs.py": 1718724477.923203, "/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/encodings/aliases.py": 1718724479.623061, "/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/encodings/__init__.py": 1718724479.616178, "/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/encodings/utf_8.py": 1718724480.430424, "/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/abc.py": 1718724477.818566, "/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/io.py": 1718724478.165237, "/Users/mpiekenbrock/opt/miniconda3/envs/blog/lib/python3.12/stat.py": 1718724478.557084, "/Users/mpiekenb

In [2]:
#| echo: false
from bokeh.plotting import figure, show, save
from bokeh.io import output_file, output_notebook
# output_notebook(hide_banner=True, verbose=False)

import numpy as np
from scipy.spatial.distance import pdist, cdist, squareform
from bokeh.plotting import figure, show
from bokeh.models import Button, CustomJS, Slider, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.layouts import row, column
from landmark import landmarks
from landmark.datasets import load_shape

In [3]:
#| echo: false
# output_notebook(hide_banner=True, verbose=False)
X = load_shape("aggregation")[:,:2]
n = len(X)
K = 15
ind, info = landmarks(X, k = len(X), full_output=True)
X = X[ind]
radii = info['radii']

from bokeh.io import output_file
output_file("/Users/mpiekenbrock/peekxc.github.io/content/posts/landmark/k_slider.html")
D = ColumnDataSource(
  dict(
    x=X[:,0], y=X[:,1],
    point_color=np.where(np.arange(n) < K, 'red', 'gray'),
    # radius=np.repeat(radii[K], n),
    radius = np.where(np.arange(n) < K, radii[K], 0.0),
    ir=radii
  )
)
ps = figure(width=400, height=400, title="Original data + Landmarks", match_aspect=True)
cg = ps.circle(
  x='x', y='y', radius='radius', fill_color='yellow',
  fill_alpha=0.10, line_color='black', line_width=0.5,
  source=D
)
sg = ps.scatter(x='x', y='y', color='point_color', source=D)

k_slider = Slider(start=2, end=len(X), value=15, step=1, title="Number of Landmarks")
cg_callback = CustomJS(args=dict(source=D), code="""
  const k = cb_obj.value
  console.log(source.data)
  const R = source.data.radius
  source.data = {
    x: source.data.x,
    y: source.data.y,
    point_color: Array.from(R, (r, i) => (i < k ? 'red' : 'gray')),
    radius: Array.from(R, (r, i) => (i < k ? source.data.ir[k-1] : 0.0)),
    ir: source.data.ir
  }
""")
k_slider.js_on_change('value', cg_callback)
ps.toolbar_location = None
p = column(k_slider, ps)
save(p)
None

In [4]:
#| echo: true
import imageio
parrot_mat = imageio.imread("/Users/mpiekenbrock/peekxc.github.io/content/posts/landmark/parrots/parrot.jpeg")
parrot_mat = parrot_mat[:,:,0]

/var/folders/0l/b3dbb2_d2bb4y3wbbfk0wt_80000gn/T/ipykernel_65069/3392017110.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  parrot_mat = imageio.imread("/Users/mpiekenbrock/peekxc.github.io/content/posts/landmark/parrots/parrot.jpeg")


In [5]:
#| echo: false
from collections import Counter
from dahuffman import HuffmanCodec

intensity_freq = Counter(np.ravel(parrot_mat))
codec = HuffmanCodec.from_frequencies(intensity_freq)
codec.print_code_table()

Bits Code              Value Symbol
   7 0000000               0 np.uint8(97)
   7 0000001               1 np.uint8(219)
   9 000001000             8 np.uint8(145)
   9 000001001             9 np.uint8(151)
   9 000001010            10 np.uint8(155)
   9 000001011            11 np.uint8(154)
   7 0000011               3 np.uint8(108)
   7 0000100               4 np.uint8(75)
   7 0000101               5 np.uint8(86)
   7 0000110               6 np.uint8(217)
   7 0000111               7 np.uint8(220)
   7 0001000               8 np.uint8(90)
   7 0001001               9 np.uint8(64)
   9 000101000            40 np.uint8(156)
  11 00010100100         164 np.uint8(14)
  12 000101001010        330 np.uint8(243)
  13 0001010010110       662 np.uint8(245)
  14 00010100101110     1326 np.uint8(248)
  14 00010100101111     1327 np.uint8(250)
  11 00010100110         166 np.uint8(6)
  11 00010100111         167 np.uint8(12)
   8 00010101             21 np.uint8(29)
   7 0001011              11

In [6]:
#| echo: true
import io
from contextlib import redirect_stdout
std_out = io.StringIO()
codec.print_code_table(std_out)
print('\n'.join(std_out.getvalue().split('\n')[:15]))

Bits Code              Value Symbol
   7 0000000               0 np.uint8(97)
   7 0000001               1 np.uint8(219)
   9 000001000             8 np.uint8(145)
   9 000001001             9 np.uint8(151)
   9 000001010            10 np.uint8(155)
   9 000001011            11 np.uint8(154)
   7 0000011               3 np.uint8(108)
   7 0000100               4 np.uint8(75)
   7 0000101               5 np.uint8(86)
   7 0000110               6 np.uint8(217)
   7 0000111               7 np.uint8(220)
   7 0001000               8 np.uint8(90)
   7 0001001               9 np.uint8(64)
   9 000101000            40 np.uint8(156)


In [7]:
code_table = codec.get_code_table()
num_bits = sum([code_table[val][0] for val in np.ravel(parrot_mat)])
print(num_bits / 8 / 1024)

138.62744140625


In [8]:
from landmark import landmarks

parrot_values = np.ravel(parrot_mat)
landmark_ind = landmarks(parrot_values[:,np.newaxis], 15, seed=np.flatnonzero(parrot_values == 128)[0])


landmark_ind = landmarks(np.arange(256)[:,np.newaxis], seed=0)

print(parrot_values[landmark_ind])

[ 58 105  99  99  83  98  96 102  69  76  94  99  96  98 100]


In [9]:
landmark_ind = landmarks(parrot_values[:,np.newaxis], 256, seed=np.flatnonzero(parrot_values == 128)[0])

enc_size = []
for k in range(2, 256):
	codeword_ind = landmark_ind[:k]
	code_mapping = np.argmin(parrot_values[:,np.newaxis] - parrot_values[codeword_ind], axis=1)
	parrot_enc = codeword_ind[code_mapping]
	codec = HuffmanCodec.from_frequencies(Counter(parrot_enc))
	code_table = codec.get_code_table()
	num_bits = sum([code_table[val][0] for val in parrot_enc])
	enc_size.append(num_bits)

enc_size = np.array(enc_size)
p = figure(width=300, height=250, title="Compression level")

p.yaxis.axis_label = "Encoding size (KiB)"
p.xaxis.axis_label = "Dictionary size (k)"
p.line(np.arange(256), enc_size / 8 / 1024)
show(p)

In [10]:
#| echo: false
import numpy as np
from landmark import landmarks
from scipy.spatial.distance import cdist, pdist, squareform
from bokeh.plotting import figure, show
from bokeh.models import SetValue, Slider, CustomJS
from bokeh.layouts import column
from bokeh.io import output_file

# base_path = '/Users/mpiekenbrock/peekxc.github.io/content/posts/landmark/'
base_path = '.'
out_path = '/Users/mpiekenbrock/peekxc.github.io/content/posts/landmark/'
output_file(out_path + "parrot_slider.html")

w,h = 316, 474
p = figure(height=w, width=h, x_range=(0,w), y_range=(0,h))

img_g = p.image_url(
  url=[base_path + '/parrots/parrot_5.jpeg'], x=w/2, y=h/2, w=w, h=h, anchor="center"
)
p.xaxis.visible = False
p.yaxis.visible = False
p.toolbar_location = None
k_slider = Slider(start=2, end=30, value=5, step=1, title="Number of Landmarks")
cg_callback = CustomJS(args=dict(source=img_g.data_source), code=f"""
  const k = cb_obj.value
  console.log(source.data)
  source.data = {{ url: ['parrots/parrot_' + k + '.jpeg']
  }}
""")
k_slider.js_on_change('value', cg_callback)
q = column(k_slider, p)
save(q)
None